# Meal Delivery Sales Analysis — Pandas & NumPy

This notebook analyzes **meal delivery orders** using **Pandas** and **NumPy**.
It covers EDA, feature engineering, missing data handling, aggregations, time analysis, merging datasets, filters, and core business KPIs.

## Data location
This project expects the files below inside the `data/` folder:
- `data/pedidos.csv`
- `data/cardapio.csv`

Note: If you extracted the datasets on macOS, you may also have a `data/__MACOSX/` folder and files like `._pedidos.csv` and `._cardapio.csv`.
These are **metadata artifacts** and should be ignored.


## 1) Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## 2) Load data

In [ ]:
DATA_DIR = "data"
PEDIDOS_PATH = os.path.join(DATA_DIR, "pedidos.csv")
CARDAPIO_PATH = os.path.join(DATA_DIR, "cardapio.csv")

print("Files in data/:\n")
for root, dirs, files in os.walk(DATA_DIR):
    level = root.replace(DATA_DIR, "").count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = "  " * (level + 1)
    for f in sorted(files):
        print(f"{subindent}{f}")

df_pedidos = pd.read_csv(PEDIDOS_PATH)
df_cardapio = pd.read_csv(CARDAPIO_PATH)

print("\nLoaded:")
print("df_pedidos shape:", df_pedidos.shape)
print("df_cardapio shape:", df_cardapio.shape)


## 3) EDA (Exploratory Data Analysis)

In [ ]:
df_pedidos.head()

In [ ]:
df_pedidos.tail()

In [ ]:
df_pedidos.info()

In [ ]:
df_pedidos.describe(include="all")

Quick checks: number of rows/columns and data types.

In [ ]:
print("Rows:", df_pedidos.shape[0])
print("Columns:", df_pedidos.shape[1])
print("\nDtypes:\n", df_pedidos.dtypes)

## 4) Feature Engineering
Create `Receita_Item` (item revenue) = `Quantidade * Preco_Unitario`.

In [ ]:
# Ensure numeric columns are numeric (coerce errors to NaN)
df_pedidos["Quantidade"] = pd.to_numeric(df_pedidos["Quantidade"], errors="coerce")
df_pedidos["Preco_Unitario"] = pd.to_numeric(df_pedidos["Preco_Unitario"], errors="coerce")

# Create item revenue
df_pedidos["Receita_Item"] = df_pedidos["Quantidade"] * df_pedidos["Preco_Unitario"]
df_pedidos.head()

## 5) Missing values handling
- Fill missing `Quantidade` with the mean
- Drop rows with missing `Preco_Unitario`

In [ ]:
df_pedidos.isna().sum()

In [ ]:
df_clean = df_pedidos.copy()

# Fill missing Quantidade with mean
qtd_mean = df_clean["Quantidade"].mean()
df_clean["Quantidade"] = df_clean["Quantidade"].fillna(qtd_mean)

# Drop rows with missing Preco_Unitario
df_clean = df_clean.dropna(subset=["Preco_Unitario"]).copy()

# Recompute Receita_Item after cleaning (keeps consistency)
df_clean["Receita_Item"] = df_clean["Quantidade"] * df_clean["Preco_Unitario"]

df_clean.info()

## 6) Aggregations by Item
- Total quantity sold per item
- Total revenue per item
- Top 5 by quantity
- Top 5 by revenue

In [ ]:
agg_item = (
    df_clean.groupby("Item")
    .agg(
        Quantidade_Total=("Quantidade", "sum"),
        Receita_Total=("Receita_Item", "sum")
    )
    .reset_index()
)

agg_item.head()

In [ ]:
top5_qty = agg_item.sort_values(by="Quantidade_Total", ascending=False).head(5)
top5_rev = agg_item.sort_values(by="Receita_Total", ascending=False).head(5)

print("Top 5 items by quantity:\n")
display(top5_qty)

print("\nTop 5 items by revenue:\n")
display(top5_rev)

## 7) Time analysis
- Convert `Data` to datetime
- Extract month
- Revenue by month
- Sales trend over time (simple plot)

In [ ]:
df_clean["Data"] = pd.to_datetime(df_clean["Data"], errors="coerce")
df_clean = df_clean.dropna(subset=["Data"]).copy()

df_clean["AnoMes"] = df_clean["Data"].dt.to_period("M").astype(str)
vendas_por_mes = df_clean.groupby("AnoMes")["Receita_Item"].sum().reset_index().sort_values("AnoMes")

vendas_por_mes.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(vendas_por_mes["AnoMes"], vendas_por_mes["Receita_Item"])
plt.xticks(rotation=45, ha="right")
plt.title("Revenue by Month")
plt.xlabel("Year-Month")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

## 8) Data integration (merge)
- Merge orders with menu using `Item`
- Revenue by category
- Category with highest revenue

In [ ]:
df_merged = pd.merge(df_clean, df_cardapio, on="Item", how="left")
df_merged.head()

In [ ]:
rev_by_cat = df_merged.groupby("Categoria")["Receita_Item"].sum().reset_index().sort_values("Receita_Item", ascending=False)
rev_by_cat

In [ ]:
top_category = rev_by_cat.iloc[0]
print("Top revenue category:")
print(top_category)

## 9) Filters & queries
Filter orders from category `Salgados` with quantity > 10.

In [ ]:
filtro = df_merged.query('Categoria == "Salgados" and Quantidade > 10')
filtro.head(20)

## 10) Business KPIs (NumPy)
- Total Revenue
- Total Items Sold
- Average Ticket (Total Revenue / number of orders)

Extra (optional): percentiles (25%, 50%, 75%) for `Preco_Unitario` and `Quantidade` using NumPy.

In [ ]:
total_revenue = float(np.sum(df_clean["Receita_Item"]))
total_items = float(np.sum(df_clean["Quantidade"]))
num_orders = int(df_clean["ID_Pedido"].nunique())
avg_ticket = total_revenue / num_orders if num_orders else np.nan

print(f"Total Revenue: {total_revenue:,.2f}")
print(f"Total Items Sold: {total_items:,.0f}")
print(f"Number of Orders: {num_orders}")
print(f"Average Ticket: {avg_ticket:,.2f}")


In [ ]:
percentis_preco = np.percentile(df_clean["Preco_Unitario"].dropna(), [25, 50, 75])
percentis_qtd = np.percentile(df_clean["Quantidade"].dropna(), [25, 50, 75])

print("Percentiles for Preco_Unitario (25%, 50%, 75%):", percentis_preco)
print("Percentiles for Quantidade (25%, 50%, 75%):", percentis_qtd)


## Next steps (ideas)
- Add more charts (top items, category mix)
- Check pricing vs `Preco_Base` (menu base price)
- Add cohort or customer-level analysis (repeat buyers)
